In [ ]:
%%sh
wget http://data.mxnet.io/data/caltech-256/caltech-256-60-train.rec
wget http://data.mxnet.io/data/caltech-256/caltech-256-60-val.rec

In [ ]:
import sagemaker

print(sagemaker.__version__)

sess= sagemaker.Session()
bucket = sess.default_bucket()

In [ ]:
prefix = 'caltech256'

s3_train_path = sess.upload_data(path='caltech-256-60-train.rec', bucket=bucket, key_prefix=prefix+'/input/train')
s3_val_path = sess.upload_data(path='caltech-256-60-val.rec',   bucket=bucket, key_prefix=prefix+'/input/validation')
s3_output = 's3://{}/{}/output'.format(bucket, prefix)

print(s3_train_path)
print(s3_val_path)
print(s3_output)

### Get the name of the image classification algorithm in our region

In [ ]:
from sagemaker.image_uris import retrieve

region = sess.boto_session.region_name    
container = retrieve('image-classification', region)
print(container)

### Configure the training job

In [ ]:
ic = sagemaker.estimator.Estimator(
    container,
    sagemaker.get_execution_role(),
    instance_count=1,
    instance_type='ml.p3.8xlarge',
    output_path=s3_output)

### Set algorithm parameters

In [ ]:
ic.set_hyperparameters(
    num_layers=50,               # Train a Resnet-18 model
    use_pretrained_model=1,      # Train from scratch
    num_classes=257,             # 256 classes + 1 clutter class
    num_training_samples=15240,  # Number of training samples
    learning_rate=0.001,
    epochs=5)

### Set dataset parameters

In [ ]:
train_data = sagemaker.TrainingInput(
    s3_train_path, 
    content_type='application/x-recordio')

validation_data = sagemaker.TrainingInput(
    s3_val_path,
    content_type='application/x-recordio')

s3_channels = {'train': train_data, 'validation': validation_data}

### Train the model

In [ ]:
ic.fit(inputs=s3_channels)

### Deploy the model

In [ ]:
ic_predictor = ic.deploy(
    initial_instance_count=1,
    instance_type='ml.t2.medium')

### Download a test image

In [ ]:
# Dog
!wget -O /tmp/test.jpg https://upload.wikimedia.org/wikipedia/commons/b/b7/LabradorWeaving.jpg
file_name = '/tmp/test.jpg'
from IPython.display import Image
Image(file_name)

### Predict test image

In [ ]:
import boto3, json
import numpy as np

runtime = boto3.Session().client(service_name='runtime.sagemaker')

with open(file_name, 'rb') as f:
    payload = f.read()
    payload = bytearray(payload)

response = runtime.invoke_endpoint(EndpointName=ic_predictor.endpoint_name, 
                                   ContentType='application/x-image', 
                                   Body=payload)

result = response['Body'].read()
result = json.loads(result)
index = np.argmax(result)
print(result[index], index)

# You should see class index 55, corresponding to the 56th class: 'dog'
# https://www.kaggle.com/jessicali9530/caltech256

### Delete endpoint

In [ ]:
ic_predictor.delete_endpoint()